# Cluster weak lensing

Galaxy cluster weak lensing is used to find the total mass inside of a galaxy cluster. Stacked cluster lensing finds the average mass of the stack. In this example we will use the `ARTsampler` to sample the posterior of a 7-dimensional model of a stacked weak lensing analysis. We also have a chain from an original MCMC analysis (with `emcee`) that we can compare against.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy.interpolate as interp
import ARTsampler
import resampler
import cluster_toolkit as ct
import scipy.stats
%matplotlib inline

In [2]:
#Plot configurations
plt.rc("text", usetex=True)
plt.rc("font", size=24, family="serif")
plt.rc("errorbar", capsize=3)

In [3]:
#Read in the chain and remove burn-in (which I only know is there for this example)
Npoints = 32*2500 #32 walkers
input_chain = np.loadtxt("chain_full_Y1_SAC_z0_l3.FORSAMPLER")[-Npoints:]
lnpost = np.loadtxt("likes_full_Y1_SAC_z0_l3.FORSAMPLER")[-Npoints:]
print("chain shape is  ", input_chain.shape)
print("lnpost shape is ", lnpost.shape)

chain shape is   (80000, 7)
lnpost shape is  (80000,)


# Step 0: set up a reconstruction of the posterior
For now, we won't use the _true_ posterior, but an ART reconstruction of the posterior.

In [ ]:
#Load in the arguments that we will use
#Note: the file was pickled with python 2.7, so we have to encode
args = pickle.load(open("z0_l3_args.p", "rb"), encoding='latin1')

Marr = args["Marr"]
bias_arr = args["bias_arr"]
args["b_spline"] = interp.interp1d(Marr, bias_arr)

def clusterWL_lnpost(params, args):
    lM, c, tau, fmis, Am, B0, Rs = params
    
    if lM < 13.0 or lM > 16.0 or c <= 1.0 or c > 15.0 or Am <= 0.7\
        or tau <= 0.0  or fmis < 0.0 or fmis > 1.0: 
        return -1e99
    if B0 < 0.0 or B0 > 2. or Rs <=0.0 or  Rs > 10.: 
        return -1e99
    LPfmis = (0.32 - fmis)**2/0.05**2
    LPtau  = (0.153 - tau)**2/0.03**2
    LPA    = (1.021459904697847 - Am)**2/0.0006116071885671028
    
    lnlike = -0.5*(LPfmis + LPtau + LPA) #Sum of the priors
    
    h = args["h"]
    z = args["z"]
    inds = args["kept_inds"]
    r = args['r'] #Mpc/h comoving
    xi_nl = args['xi_nl']
    b_spline = args["b_spline"]
    Redges = args["Redges"] #Mpc/h comoving
    Rp = args['Rp'] #Mpc/h comoving; projected on the sky
    Rlam = args["Rlam"] #Mpc/h comoving
    lam = Rlam**5 * 100. #richness
    Sigma_crit_inv = args["Sigma_crit_inv"] #pc^2/hMsun comoving
    M = 10**lM
    Omega_m = 0.3 #CHECK THIS
    xi_nfw   = ct.xi.xi_nfw_at_r(r, M, c, Omega_m)
    bias = b_spline(M)
    xi_2halo = ct.xi.xi_2halo(bias, xi_nl)
    xi_hm    = ct.xi.xi_hm(xi_nfw, xi_2halo)
    Sigma  = ct.deltasigma.Sigma_at_R(Rp, r, xi_hm, M, c, Omega_m)
    DeltaSigma = ct.deltasigma.DeltaSigma_at_R(Rp, Rp, Sigma, M, c, Omega_m)
    Rmis = tau*Rlam #Mpc/h
    Sigma_mis  = ct.miscentering.Sigma_mis_at_R(Rp, Rp, Sigma, M, c, 
                                                Omega_m, Rmis, kernel="gamma")
    DeltaSigma_mis = ct.miscentering.DeltaSigma_mis_at_R(Rp, Rp, Sigma_mis)
    #Note: here, Rs is Mpc physical but Rp is Mpc/h comoving
    boost = ct.boostfactors.boost_nfw_at_R(Rp, B0, Rs*h*(1+z))
    full_Sigma = (1-fmis)*Sigma + fmis*Sigma_mis
    full_DeltaSigma = (1-fmis)*DeltaSigma + fmis*DeltaSigma_mis #miscentering
    full_DeltaSigma *= Am #multiplicative bias
    full_DeltaSigma /= boost #boost factor
    full_DeltaSigma /= (1-full_Sigma*Sigma_crit_inv) #reduced shear
    ave_DeltaSigma = ct.averaging.average_profile_in_bins(Redges, Rp, full_DeltaSigma)
    
    #print("ave DeltaSigma is:\n\t",ave_DeltaSigma)
    #print("SCI: ", Sigma_crit_inv)
    #print("halo bias: ", bias)
    DS_data = args["DeltaSigma"]
    Cov = args["Cov"]

    #Convert to Msun/pc^2 physical
    X = (DS_data - ave_DeltaSigma[inds]*h*(1+z)**2)
    lnlike += -0.5*np.dot(X, np.linalg.solve(Cov, X))
    
    Bp1 = args['Bp1']
    Bcov = args['Bcov']
    #Note: here, Rs is Mpc physical and Rb is the same
    boost_model = ct.boostfactors.boost_nfw_at_R(args['Rb'], B0, Rs)
    #print("boost model:", boost_model)
    Xb = Bp1 - boost_model
    lnlike += -0.5*np.dot(Xb, np.linalg.solve(Bcov, Xb))  
    return lnlike

In [ ]:
params = np.array([14.662047841051441, 4.5, 0.153, 0.32, 1.02, 0.07, 2.49])
print(clusterWL_lnpost(params, args)) #answer = -9.33e+02
print(clusterWL_lnpost(input_chain[-1], args))
print(lnpost[-1])

# Step 1: set up the ART sampler
In order to set up the sampler, we need to guess the mean and covariance of the target distribution. I know roughly what the means should be, but for the covariance I'm just going to guess a Gaussian covariance with no correlations (i.e. diagonal), even though I know this is wrong.

In [ ]:
#Parameters are: log{10}(M), concentration, tau_{miscentering}, f_{miscentering}
#A_m, B_0, R_s (see McClintock et al. 2019 for details)
#guess_mean = np.array([14.1, 6., .1, 0.25, 1.02, 0.2, 1.5])
#guess_cov = np.diag(np.var(input_chain, 0))
prior_volume = np.array([[13., 16.], #log10 M
                        [1., 15.], #concentration
                        [0.05, 0.4], #tau
                        [0.05, 0.6], #fmis
                        [0.88, 1.2], #Am
                        [0.1, 1.0], #B0
                        [0.1, 1.0]]) #Rs

In [ ]:
prior_distributions = [scipy.stats.uniform(loc=13, scale=3), 
                       scipy.stats.uniform(loc=1, scale=14), 
                       scipy.stats.norm(loc=0.153, scale=0.03), 
                       scipy.stats.norm(loc=0.32, scale=0.05), 
                       scipy.stats.norm(loc=1.021459904697847, scale=0.024730693248817406), 
                       scipy.stats.uniform(loc=0.1, scale=0.9), 
                       scipy.stats.uniform(loc=0.1, scale=0.9)]

In [ ]:
sampler = ARTsampler.ARTsampler(prior_distributions, clusterWL_lnpost, args, 
                                quiet=False, scale=7,
                                Ntraining_points=200, Nburn = 1000, Nsteps=2000, max_iter=1)

In [ ]:
training_points = sampler.get_training_points()
plt.scatter(training_points[:,0], training_points[:,1], c='k', s=10, zorder=1)

plt.scatter(input_chain[:5000,0], input_chain[:5000,1], marker='.', c='b', 
            alpha=0.2, s=0.5, zorder=0)
plt.xlabel(r"$\log_{10}M_{\rm 200b}$")
plt.ylabel(r"$c_{\rm 200b}$")

# Step 2: do some iterations
Let's do two iterations to see how it does.

In [ ]:
ind = 0
while sampler.single_iteration():
    print("On iteration {0}".format(ind))
    ind += 1
    continue

In [ ]:
training_points = sampler.get_training_points()
plt.scatter(training_points[:,0], training_points[:,1], c='k', s=10, zorder=1)

samples = sampler.get_samples().T
plt.scatter(samples[0], samples[1], marker='.', c='r', alpha=0.2, s=0.5, zorder=0)

plt.scatter(input_chain[:10000,0], input_chain[:10000,1], marker='.', c='b', alpha=0.2, s=0.5)
plt.xlabel(r"$x$")
plt.ylabel(r"$y$")

In [ ]:
for i in range(0, 2):
    training_points = sampler.get_training_points(i)
    plt.scatter(training_points[:,0], training_points[:,1], s=10, label=i)
plt.legend()